In [1]:
# 将小波变换得到的图像通过CNN提取特征，保存全连接层的2048维视觉向量

In [2]:
import tensorflow as tf
import h5py
import sys
import random
import re
import matplotlib.pyplot as plt
import numpy as np
import os
import operator

from numpy import *  
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam,SGD
from keras.models import load_model,Model
from sklearn import manifold
from sklearn.cluster import KMeans

Using TensorFlow backend.


In [3]:
# 载入数据
print('当前工作目录：')
print(os.getcwd())

x_train = np.load(r".\data\train\x_train.npy")
y_train = np.load(r".\data\train\y_train.npy")
x_test = np.load(r".\data\test\x_test.npy")
y_test = np.load(r".\data\test\y_test.npy")

当前工作目录：
D:\Code\jupyter_notebook\TensorFlow\2021.5.7


In [4]:
# 训练集处理
x_train = x_train.reshape(-1,64,64,1)/255.0
# 换one hot格式
y_train = np_utils.to_categorical(y_train,num_classes=3)

In [5]:
# 测试集处理
x_test = x_test.reshape(-1,64,64,1)/255.0
# 换one hot格式
y_test = np_utils.to_categorical(y_test,num_classes=4)

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(6000, 64, 64, 1)
(6000, 3)
(8000, 64, 64, 1)
(8000, 4)


In [7]:
# 定义CNN网络模型获取可见类2048维特征

In [ ]:
model = Sequential()

#卷积层1（input_shape：（长*宽*通道数） filter：卷积核个数  kernel_size 卷积窗口大小  strides 步长 padding padding方式 same/valid activation 激活函数）
model.add(Convolution2D(
    input_shape = (64,64,1),
    filters = 32,
    kernel_size = 5,
    strides = 1,
    padding = 'same',
    activation = 'relu',
))

#池化层1
model.add(MaxPooling2D(
    pool_size = 2,
    strides = 2,
    padding = 'same',
))

# 卷积层2
model.add(Convolution2D(filters=64,
                        kernel_size=5,
                        strides=1,
                        padding='same',
                        activation='relu'))

# 池化层2
model.add(MaxPooling2D(
    pool_size = 2,
    strides = 2,
    padding = 'same',
))

# 把池化层2的输出output展开为1维
model.add(Flatten())

# 第一个全连接层
model.add(Dense(4096,activation='relu'))
# model.add(Dense(2048,activation='relu'))
model.add(Dense(2048,activation='relu'))

# Dropout
model.add(Dropout(0.25))

# 第二个全连接层
model.add(Dense(3,activation='softmax'))

# 定义优化器
adam = Adam(lr=1e-4)

# 定义优化器,loss function,训练过程中计算准确率
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

# 训练模型
model.fit(x_train,y_train,batch_size=100,epochs=25)

# 评估模型
loss,accuracy = model.evaluate(x_train,y_train)


#保存模型
model.save('model.h5')
print(model.summary())
print('train loss',loss)
print('train accuracy',accuracy)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/25
6000/6000 [==============================] - 9s 1ms/step - loss: 0.3951 - categorical_accuracy: 0.8277
Epoch 2/25
6000/6000 [==============================] - 6s 923us/step - loss: 9.1226e-04 - categorical_accuracy: 0.9998
Epoch 3/25
6000/6000 [==============================] - 6s 924us/step - loss: 1.5827e-04 - categorical_accuracy: 1.0000
Epoch 4/25
6000/6000 [==============================] - 6s 920us/step - loss: 1.0888e-04 - categorical_accuracy: 1.0000
Epoch 5/25
6000/6000 [==============================] - 6s 925us/step - loss: 8.3971e-05 - categorical_accuracy: 1.0000
Epoch 6/25
6000/6000 [==============================] - 5s 899us/step - loss: 6.2000e-05 - categorical_accuracy: 1.0000
Epoch 7/25
6000/6000 [==============================] - 5s 904us/step 

In [9]:
# 载入训练好的模型

model = load_model('model.h5')



# 获取最后一个全连接层的输出
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(str('dense_1')).output)

In [10]:
x_train = np.load(r".\data\train\x_train.npy")
x_train = x_train.reshape(-1,64,64,1)/255.0
x_train_2048 = intermediate_layer_model.predict(x_train)
print(x_train_2048.shape)

(6000, 2048)


In [11]:
# #单一故障分布
# X_seen = x_train_2048
# Y_seen = np.load('./data/train/y_train.npy')
# n_samples, n_features = X_seen.shape

# '''t-SNE'''
# tsne = manifold.TSNE(n_components=2, init='pca', random_state=501)
# X_tsne = tsne.fit_transform(X_seen)

# print("Org data dimension is {}.  Embedded data dimension is {}".format(X_seen.shape[-1], X_tsne.shape[-1]))

# '''嵌入空间可视化'''
# x_min, x_max = X_tsne.min(0), X_tsne.max(0)
# X_norm = (X_tsne - x_min) / (x_max - x_min)  # 归一化
# plt.figure(figsize=(8, 8))
# for i in range(X_norm.shape[0]):
#     plt.text(X_norm[i, 0], X_norm[i, 1], str(Y_seen[i]), color=plt.cm.Set1(Y_seen[i][0]),fontdict={'weight': 'bold', 'size': 9})
# plt.xticks([])
# plt.yticks([])
# plt.show()

In [12]:
y_train = np.load(r".\data\train\y_train.npy")
y_train_oh = np_utils.to_categorical(y_train,num_classes=3)
print(y_train_oh.shape)

(6000, 3)


In [13]:
x_test = np.load(r".\data\test\x_test.npy")
x_test = x_test.reshape(-1,64,64,1)/255.0
x_test_2048 = intermediate_layer_model.predict(x_test)
print(x_test_2048.shape)

(8000, 2048)


In [14]:
# #复合故障分布
# X_unseen = x_test_2048
# Y_unseen = np.load('./data/test/y_test.npy')
# n_samples, n_features = X_unseen.shape

# '''t-SNE'''
# tsne = manifold.TSNE(n_components=2, init='pca', random_state=501)
# X_tsne = tsne.fit_transform(X_unseen)

# print("Org data dimension is {}.  Embedded data dimension is {}".format(X_unseen.shape[-1], X_tsne.shape[-1]))

# '''嵌入空间可视化'''
# x_min, x_max = X_tsne.min(0), X_tsne.max(0)
# X_norm = (X_tsne - x_min) / (x_max - x_min)  # 归一化
# plt.figure(figsize=(8, 8))
# for i in range(X_norm.shape[0]):
#     plt.text(X_norm[i, 0], X_norm[i, 1], str(Y_unseen[i]), color=plt.cm.Set1(Y_unseen[i][0]),fontdict={'weight': 'bold', 'size': 9})
# plt.xticks([])
# plt.yticks([])
# plt.show()

In [15]:
y_test = np.load(r".\data\test\y_test.npy")
y_test_oh = np_utils.to_categorical(y_test,num_classes=4)
print(y_test_oh.shape)

(8000, 4)


In [16]:
np.save('./data/train/x_train_2048', x_train_2048)
np.save('./data/train/y_train_oh', y_train_oh)
np.save('./data/test/x_test_2048', x_test_2048)
np.save('./data/test/y_test_oh', y_test_oh)

In [17]:
print(x_train_2048.shape)
print(y_train_oh.shape)
print(x_test_2048.shape)
print(y_test_oh.shape)

(6000, 2048)
(6000, 3)
(8000, 2048)
(8000, 4)


In [ ]:
# ----------------------数据预处理已完成 -----------------------------#